Βήμα 1: Συλλογή δεδομένων

Συλλογή εγγράφων από ένα url της wikipedia και εκτύπωσή τους στην οθόνη του χρήστη

In [1]:
import requests
from bs4 import BeautifulSoup as bs

url = "https://en.wikipedia.org/wiki/Enron"
page = requests.get(url)
soup = bs(page.text, 'html.parser')

content = soup.find(id="bodyContent")
text = content.get_text()
print(text)





From Wikipedia, the free encyclopedia


American energy company
This article is about the corporation. For the documentary, see Enron: The Smartest Guys in the Room. For the play, see Enron (play). For the accounting scandal, see Enron scandal.


Enron CorporationLogo (designed by Paul Rand) used from 1997 to 2001. This was Rand's final major logo before his death in November 1996.Company typePublicTraded asOTC Pink: ENRNQNYSE: ENEIndustryEnergyPredecessorInterNorth (Northern Natural Gas Company)Houston Natural Gasmerger in 1985FoundedJuly 16, 1985; 39 years ago (1985-07-16) in Omaha, Nebraska, U.S.FounderKenneth Lay (for the Houston National Gas branch)DefunctMarch 1, 2007; 17 years ago (2007-03-01)November 28, 2016; 8 years ago (2016-11-28) (as Enron Creditors Recovery Corporation)FateChapter 11 bankruptcy (as an effect of accounting fraud)SuccessorDynegyPrisma Energy InternationalJ.M. Hansen CorporationHeadquarters1400 Smith StreetHouston, Texas, United StatesArea servedUnited S

Βήμα 2: Προεπεξεργασία κειμένου (Text Processing)

Διατέλεση τεχνικών προεπεξεργασίας κειμένου:               


α) Tokenization: Διάσπαση του κειμένου σε μικρότερα τμήματα(λέξεις, προτάσεις ή και χαρακτήρες).                                                  
β) Stemming: Αφαίρεση των καταλήξεων από τις λέξεις, χωρίς να λαμβάνεται υπόψην η γραμματική ή η σημασιολογία.               
γ) Lemmatization: Ίδια λειτουργία με το stemming, διαφέρει στο ότι είναι πιο ακριβής και χρησιμοποιεί λεξικά για να μετατρέπει τις λέξεις στις λεξικές ή βασικές τους μορφές, λαμβάνοντας υπόψη τη γραμματική και τη σημασία τους.           
δ) stop-words removal: Η αφαίρεση τους αφορά τη διαγραφή κοινών λέξεων που δεν προσθέτουν σημασιολογική αξία στο κείμενο(π.χ. και, να κ.λπ).
ε) Αφαίρεση ειδικών χαρακτήρων: Διαγράφονται χαρακτήρες οι οποίοι δεν προσφέρουν  σημασία για την ανάλυση του κειμένου(π.χ. @, #, $ κ.λπ).

In [5]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re

tokens = word_tokenize(text)
tokens = [re.sub(r'\W', '', token).lower() for token in tokens if re.sub(r'\W', '', token)]
print("Tokens:", tokens[:100])

stop_words = set(stopwords.words('english'))
tokens = [token for token in tokens if token not in stop_words]

stemmer = PorterStemmer()
stemmed_tokens = [stemmer.stem(token) for token in tokens]

lemmatizer = WordNetLemmatizer()
lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

print("Lemmatized Tokens:", lemmatized_tokens[:100])

print("Stemmed Tokens:", stemmed_tokens[:100])

Tokens: ['from', 'wikipedia', 'the', 'free', 'encyclopedia', 'american', 'energy', 'company', 'this', 'article', 'is', 'about', 'the', 'corporation', 'for', 'the', 'documentary', 'see', 'enron', 'the', 'smartest', 'guys', 'in', 'the', 'room', 'for', 'the', 'play', 'see', 'enron', 'play', 'for', 'the', 'accounting', 'scandal', 'see', 'enron', 'scandal', 'enron', 'corporationlogo', 'designed', 'by', 'paul', 'rand', 'used', 'from', '1997', 'to', '2001', 'this', 'was', 'rand', 's', 'final', 'major', 'logo', 'before', 'his', 'death', 'in', 'november', '1996company', 'typepublictraded', 'asotc', 'pink', 'enrnqnyse', 'eneindustryenergypredecessorinternorth', 'northern', 'natural', 'gas', 'company', 'houston', 'natural', 'gasmerger', 'in', '1985foundedjuly', '16', '1985', '39', 'years', 'ago', '19850716', 'in', 'omaha', 'nebraska', 'usfounderkenneth', 'lay', 'for', 'the', 'houston', 'national', 'gas', 'branch', 'defunctmarch', '1', '2007', '17', 'years', 'ago', '20070301']
Lemmatized Tokens: [

Βήμα 3: Ευρετήριο (Indexing)

Δημιουργία ενός ανεστραμμένου ευρετηρίου με την χρήση των lemmatized tokens από το προηγούμενο βημα.                    
Αποτέλεσμα είναι μια δομή δεδομένων που χαρτογραφεί κάθε μοναδικό token στις θέσεις όπου εμφανίζεται στη λίστα.    Αναλυτικά:                                                    
Δημιουργείται ένα κενό λεξικό, το οποίο θα λειτουργήσει ως το ανεστραμμένο ευρετήριο. Το κλειδί κάθε εισόδου θα είναι ένα μοναδικό token, και η τιμή του θα είναι μια λίστα με τις θέσεις όπου εμφανίζεται το token στη λίστα lemmatized_tokens.    
Στην συνέχεια μέσα στο loop (με την enumerate() επαναλαμβάνεται κάθε στοιχείο (token) στη λίστα lemmatized_tokens.) ελέγχεται αν το τρέχον token δεν υπάρχει ήδη ως κλειδί στο λεξικό και αν δεν υπάρχει προστίθεται στο inverted_index με αρχική τιμή μια κενή λίστα.Στην συνέχεια η τρέχουσα θέση (index) του token στη λίστα lemmatized_tokens προστίθεται στη λίστα τιμών που σχετίζεται με αυτό το token στο inverted_index.                                               Τέλος, γίνεται η εκτύπωση του ανεστραμμένου ευρετηρίου όπου σε αυτή την περίπτωση εμφανίζονται μόνο οι πρώτες 5 θέσεις (v[:5]) όπου εμφανίζεται κάθε token.                       

In [7]:
inverted_index = {}
for index, token in enumerate(lemmatized_tokens):
    if token not in inverted_index:
        inverted_index[token] = []
    inverted_index[token].append(index)

print("Inverted Index:", {k: v[:5] for k, v in inverted_index.items()})

Inverted Index: {'wikipedia': [0], 'free': [1, 5219, 6423], 'encyclopedia': [2, 6921, 6989], 'american': [3, 159, 6416, 7328, 8174], 'energy': [4, 90, 146, 160, 331], 'company': [5, 46, 163, 180, 199], 'article': [6, 2171, 2700, 3558, 8774], 'corporation': [7, 82, 158, 249, 520], 'documentary': [8, 3446, 5881, 8439], 'see': [9, 15, 20, 3795, 5869], 'enron': [10, 16, 21, 23, 79], 'smartest': [11, 2095, 5876, 6618, 6655], 'guy': [12, 2096, 4040, 4042, 5877], 'room': [13, 2097, 5878, 6620, 6657], 'play': [14, 17, 3441, 5915, 8434], 'accounting': [18, 87, 228, 245, 273], 'scandal': [19, 22, 233, 241, 257], 'corporationlogo': [24], 'designed': [25, 3935], 'paul': [26, 2045, 6583, 7197, 7616], 'rand': [27, 31, 2046, 6584], 'used': [28, 2468, 2869, 2999, 3626], '1997': [29, 1421, 1506, 2031, 4951], '2001': [30, 186, 217, 289, 1693], 'final': [32, 2040], 'major': [33, 192, 481, 886, 1083], 'logo': [34, 2037, 2038, 6619], 'death': [35, 2047, 5908, 7890, 8045], 'november': [36, 72, 299, 1001, 32

Βήμα 4: Μηχανή αναζήτησης (Search Engine)

Μέρος Α: Επεξεργασία ερωτήματος (Query Processing)                                                                  
Δημιουργία της κλάσης  QueryProcessor για την επεξεργασία ερωτημάτων με τη χρήση λογικών (Boolean) πράξεων (AND, OR, και NOT).                                                                                                          
Αναλυτικά:       

    1) Γίνεται λήψη δεδομένων από το ανεστραμμένο ευρετήριο και αποθήκευση τους (συνάρτηση _init_).                 
    2) Tokenizing(συνάρτηση   tokenize)                                                                                                     
        α)Γίνεται ο χωρισμός ενός ερωτήματος χρήστη σε tokens.              
        β)Χρησιμοποιούνται κανονικές εκφράσεις για την εξαγωγή λέξεων, αγνοώντας κεφαλαία/πεζά και ειδικούς χαρακτήρες.
    3)Ανάλυση του ερωτήματος και επιλογή της σωστής Boolean λειτουργίας(συνάρτηση search):
        α)Λειτουργία AND: Κοινά έγγραφα μεταξύ δύο συνόλων λέξεων.
        β)Λειτουργία OR: Συνδυασμός εγγράφων που περιέχουν είτε τη μία είτε την άλλη λέξη.
        γ)Λειτουργία ΝΟΤ: Αφαιρεί έγγραφα που περιέχουν τις λέξεις μετά το NOT.
    4)Χρήση βοηθητικών συναρτήσεων:
        α)_simple_search: Βρίσκει έγγραφα που περιέχουν όλα τα tokens.
        β)_and_search: Διασταύρωση (AND) αποτελεσμάτων.
        γ)_or_search: Ένωση (OR) αποτελεσμάτων.
        δ)_not_search: Αφαίρεση (NOT) αποτελεσμάτων.  

Μέρος Β: Κατάταξη αποτελεσμάτων (Ranking):

Πραγματοποιείται ενσωμάτωση ορισμένων αλγορίθμων κατάταξης:

    α) TF-IDF Model : Μέτρηση της σημασίας ενός όρου σε ένα έγγραφο αναλογικά με την παρουσία του σε όλα τα έγγραφα.
    Δημιουργείται μια κλάση TFIDF μέσα στην οποία πραμγατοποιούνται οι εξής λειτουργίες:
        i)Αρχικοποίηση της _init_:
             Αποθηκευση των εγγράφων (self.documents) και υπολογισμός της IDF (Inverse Document Frequency) για όλους  τους όρους(συνάρτηση self.compute_idf).
        ii)Υπολογισμός TF (Term Frequency)
             Υπολογίζεται πόσες φορές εμφανίζεται ένας όρος σε ένα έγγραφο και τον κανονικοποιεί με βάση το μέγεθος του εγγράφου(συνάρτηση compute_tf).
        iii)Υπολογισμός IDF
             Υπολογισμός της σπανιότητας του κάθε όρου σε όλα τα έγγραφα(συνάρτηση compute_idf).
        iv)Υπολογισμός TF-IDF
             Συνδυασμός TF και IDF για κάθε όρο στο έγγραφο(συνάρτηση compute_tfidf).
        v)Κατάταξη Εγγράφων
             Βασίζεται στο άθροισμα των TF-IDF τιμών των όρων του ερωτήματος που εμφανίζονται στα έγγραφα.
    
    β)Vector Space Model: Βασίζεται στην αναπαράσταση τόσο των εγγράφων όσο και του ερωτήματος ως διανύσματα σε έναν πολυδιάστατο χώρο.
    Δημιουργείται μια κλάση VectorSpaceModel μέσα στην οποία πραγματοποιούνται οι εξής λειτουργίες:
        i)Υπολογισμός Συνημιτόνου(συνάρτηση compute_cosine_similarity):
            Γίνεται μέτρηση της γωνίας μεταξύ των διανυσμάτων του ερωτήματος και του εγγράφου. Όσο μικρότερη είναι η γωνία, τόσο πιο συναφής είναι η μέτρηση.
        ii)Κατάταξη Εγγράφων:
            Υπολογισμός της συσχέτισης συνημιτόνου μεταξύ του διανύσματος του ερωτήματος και των εγγράφων(συνάρτηση rank).

    γ)Okapi BM25: Πρόκειται για ένα προηγμένο μοντέλο που βελτιώνει την απλή συχνότητα όρων, λαμβάνοντας υπόψη το μήκος του εγγράφου και τη σημασία της συχνότητας.
     Δημιουργείται μια κλάση Okapi BM25 μέσα στην οποία πραμγατοποιούνται οι εξής λειτουργίες:
        i)Υπολογισμός IDF(συνάρτηση compute_idf)
            Η IDF για το BM25 διαφέρει από το TF-IDF. Το ΒΜ25 ενσωματώνει μια πιο σύνθετη εξίσωση για καλύτερη κανονικοποίηση.
        ii)Υπολογισμός του BM25 Score (συνάρτηση compute_bm25)
            Λαμβάνεται υπόψη η συχνότητα όρων, η σπανιότητα και το μήκος του εγγράφου
        iii)Κατάταξη Εγγράφων
            Υπολογίζεται και ταξινομείται το BM25 score για κάθε έγγραφο (συνάρτηση rank).


        



        



In [ ]:
import requests
import nltk
from bs4 import BeautifulSoup as bs
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re
from collections import defaultdict
import math
from sklearn.metrics import precision_score, recall_score, f1_score
from collections import defaultdict
import math
from sklearn.metrics import precision_score, recall_score, f1_score




class QueryProcessor:
    def __init__(self, data):
        self.data = data

    def tokenize(self, query):
        return re.findall(r'\w+', query.lower())

    def search(self, query):
        tokens = self.tokenize(query)
        results = set(self.data)
        
        if 'and' in tokens:
            and_index = tokens.index('and')
            left_tokens = tokens[:and_index]
            right_tokens = tokens[and_index+1:]
            results = self._and_search(left_tokens, right_tokens)
        elif 'or' in tokens:
            or_index = tokens.index('or')
            left_tokens = tokens[:or_index]
            right_tokens = tokens[or_index+1:]
            results = self._or_search(left_tokens, right_tokens)
        elif 'not' in tokens:
            not_index = tokens.index('not')
            left_tokens = tokens[:not_index]
            right_tokens = tokens[not_index+1:]
            results = self._not_search(left_tokens, right_tokens)
        else:
            results = self._simple_search(tokens)
        
        return results

    def _simple_search(self, tokens):
        return {item for item in self.data if all(token in item.lower() for token in tokens)}

    def _and_search(self, left_tokens, right_tokens):
        left_results = self._simple_search(left_tokens)
        right_results = self._simple_search(right_tokens)
        return left_results & right_results

    def _or_search(self, left_tokens, right_tokens):
        left_results = self._simple_search(left_tokens)
        right_results = self._simple_search(right_tokens)
        return left_results | right_results

    def _not_search(self, left_tokens, right_tokens):
        left_results = self._simple_search(left_tokens)
        right_results = self._simple_search(right_tokens)
        return left_results - right_results

# Create a TF-IDF model
class TFIDF:
    def __init__(self, documents):
        self.documents = documents
        self.idf = self.compute_idf()

    def compute_tf(self, document):
        tf = defaultdict(float)
        for term in document:
            tf[term] += 1
        for term in tf:
            tf[term] /= len(document)
        return tf

    def compute_idf(self):
        idf = defaultdict(float)
        total_documents = len(self.documents)
        for document in self.documents.values():
            for term in set(document):
                idf[term] += 1
        for term in idf:
            idf[term] = math.log(total_documents / (1 + idf[term]))
        return idf

    def compute_tfidf(self, document):
        tf = self.compute_tf(document)
        tfidf = defaultdict(float)
        for term in tf:
            tfidf[term] = tf[term] * self.idf[term]
        return tfidf

    def rank(self, query):
        query_tokens = word_tokenize(query.lower())
        query_tokens = [re.sub(r'\W', '', token) for token in query_tokens if re.sub(r'\W', '', token)]
        query_tokens = [token for token in query_tokens if token not in stop_words]
        query_tokens = [lemmatizer.lemmatize(token) for token in query_tokens]

        query_tfidf = self.compute_tfidf(query_tokens)
        scores = defaultdict(float)

        for doc_id, document in self.documents.items():
            doc_tfidf = self.compute_tfidf(document)
            for term in query_tfidf:
                if term in doc_tfidf:
                    scores[doc_id] += query_tfidf[term] * doc_tfidf[term]

        ranked_docs = sorted(scores.items(), key=lambda item: item[1], reverse=True)
        return ranked_docs
class VectorSpaceModel:
    def __init__(self, documents):
        self.documents = documents
        self.tfidf = TFIDF(documents)

    def compute_cosine_similarity(self, vec1, vec2):
        intersection = set(vec1.keys()) & set(vec2.keys())
        numerator = sum([vec1[x] * vec2[x] for x in intersection])

        sum1 = sum([vec1[x] ** 2 for x in vec1.keys()])
        sum2 = sum([vec2[x] ** 2 for x in vec2.keys()])
        denominator = math.sqrt(sum1) * math.sqrt(sum2)

        if not denominator:
            return 0.0
        else:
            return float(numerator) / denominator

    def rank(self, query):
        query_tokens = word_tokenize(query.lower())
        query_tokens = [re.sub(r'\W', '', token) for token in query_tokens if re.sub(r'\W', '', token)]
        query_tokens = [token for token in query_tokens if token not in stop_words]
        query_tokens = [lemmatizer.lemmatize(token) for token in query_tokens]

        query_tfidf = self.tfidf.compute_tfidf(query_tokens)
        scores = defaultdict(float)

        for doc_id, document in self.documents.items():
            doc_tfidf = self.tfidf.compute_tfidf(document)
            scores[doc_id] = self.compute_cosine_similarity(query_tfidf, doc_tfidf)

        ranked_docs = sorted(scores.items(), key=lambda item: item[1], reverse=True)
        return ranked_docs
class OkapiBM25:
    def __init__(self, documents, k1=1.5, b=0.75):
        self.documents = documents
        self.k1 = k1
        self.b = b
        self.avg_doc_len = self.compute_avg_doc_len()
        self.idf = self.compute_idf()

    def compute_avg_doc_len(self):
        total_length = sum(len(doc) for doc in self.documents.values())
        return total_length / len(self.documents)

    def compute_idf(self):
        idf = defaultdict(float)
        total_documents = len(self.documents)
        for document in self.documents.values():
            for term in set(document):
                idf[term] += 1
        for term in idf:
            idf[term] = math.log((total_documents - idf[term] + 0.5) / (idf[term] + 0.5) + 1)
        return idf

    def compute_bm25(self, document, query_tokens):
        doc_len = len(document)
        score = 0.0
        for term in query_tokens:
            if term in document:
                term_freq = document.count(term)
                numerator = self.idf[term] * term_freq * (self.k1 + 1)
                denominator = term_freq + self.k1 * (1 - self.b + self.b * (doc_len / self.avg_doc_len))
                score += numerator / denominator
        return score

    def rank(self, query):
        query_tokens = word_tokenize(query.lower())
        query_tokens = [re.sub(r'\W', '', token) for token in query_tokens if re.sub(r'\W', '', token)]
        query_tokens = [token for token in query_tokens if token not in stop_words]
        query_tokens = [lemmatizer.lemmatize(token) for token in query_tokens]


        scores = defaultdict(float)
        for doc_id, document in self.documents.items():
            scores[doc_id] = self.compute_bm25(document, query_tokens)
            ranked_docs = sorted(scores.items(), key=lambda item: item[1], reverse=True)
        return ranked_docs
    
#Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

url = "https://en.wikipedia.org/wiki/Enron"
page = requests.get(url)
soup = bs(page.text, 'html.parser')

#Extract text from the main content of the webpage
content = soup.find(id="bodyContent")
text = content.get_text()

#Tokenization
tokens = word_tokenize(text)

#Remove special characters and convert to lowercase
tokens = [re.sub(r'\W', '', token).lower() for token in tokens if re.sub(r'\W', '', token)]

#Remove stop words
stop_words = set(stopwords.words('english'))
tokens = [token for token in tokens if token not in stop_words]

#Stemming
stemmer = PorterStemmer()
stemmed_tokens = [stemmer.stem(token) for token in tokens]

#Lemmatization
lemmatizer = WordNetLemmatizer()
lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

#print("Lemmatized Tokens:", lemmatized_tokens[:100])
#print("Stemmed Tokens:", stemmed_tokens[:100])

# Create inverted index
inverted_index = {}
for index, token in enumerate(lemmatized_tokens):
    if token not in inverted_index:
        inverted_index[token] = []
    inverted_index[token].append(index)
#print("Inverted Index:", {k: v[:5] for k, v in inverted_index.items()})  # Print first 5 entries for brevity
# Save the inverted index to a file
qp =QueryProcessor(inverted_index)
documents = {
    0: inverted_index,
}
tfidf = TFIDF(documents)
vsm = VectorSpaceModel(documents)

bm25 = OkapiBM25(inverted_index)

while True:
    print("\nChoose an option:")
    print("1. TF-IDF Ranking")
    print("2. Boolean Retrieval")
    print("3. Vector Space Model")
    print("4. Okapi BM25 Ranking")
    print("5. Exit")
    choice = input("Enter your choice (1-5): ")

    if choice == '1':
        query = input("Enter your query for TF-IDF Ranking: ")
        ranked_docs = tfidf.rank(query)
        print(f"Ranked documents for '{query}':", ranked_docs)
    elif choice == '2':
        query = input("Enter your query for Boolean Retrieval: ")
        qp =QueryProcessor(inverted_index)
        results = qp.search(query)
        print(f"Search results for '{query}':", results)
    elif choice == '3':
        query = input("Enter your query for Vector Space Model: ")
        ranked_docs = vsm.rank(query)
        print(f"Ranked documents for '{query}':", ranked_docs)
    elif choice == '4':
        query = input("Enter your query for Okapi BM25 Ranking: ")
        ranked_docs = bm25.rank(query)
        print(f"Ranked documents for '{query}':", ranked_docs)
    elif choice == '5':
        print("Exiting...")
        break
    else:
        print("Invalid choice. Please try again.")






[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Costa.000\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Costa.000\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Costa.000\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



Choose an option:
1. TF-IDF Ranking
2. Boolean Retrieval
3. Vector Space Model
4. Okapi BM25 Ranking
5. Exit


Βήμα 5: Αξιολόγηση συστήματος


Πραγματοποιείται αξιολόγηση της απόδοσης της μηχανής αναζήτησης χρησιμοποιώντας τυποποιημένες μετρικές, οι οποίες μετρούν πόσο αποτελεσματικά το σύστημα ανακτά σχετικά έγγραφα.                

Εκτελώντας τον παρακάτω κώδικα  υπολογίζουμε τις εξής μετρικές αξιολόγησης:

α)Ακρίβεια (Precision):
Μέτρηση της αναλογίας των σχετικών εγγράφων που ανακτήθηκαν από τα συνολικά ανακτημένα.

β)Ανάκληση (Recall):
Μέτρηση της αναλογίας των σχετικών εγγράφων που ανακτήθηκαν σε σχέση με όλα τα σχετικά έγγραφα.

γ)F1-Score:
Ο συνδυασμός ακρίβειας και ανάκλησης σε ένα μέτρο.

Αναλυτικά οι παραπάνω αυτές μετρικές υπολογίζονται όλες μέσα στην συνάρτηση evaluate_search_engine παρακάτω

1)Αρχικοποίηση Λιστών:

y_true: Λίστα που θα περιέχει τις πραγματικές τιμές (1 αν το έγγραφο είναι σχετικό, 0 αν δεν είναι).            
y_pred: Λίστα που θα περιέχει τις προβλεπόμενες τιμές (1 αν το έγγραφο ανακτήθηκε, 0 αν δεν ανακτήθηκε).

2)Επεξεργασία Ερωτημάτων:

    α)Για κάθε ερώτημα (query) και τα αντίστοιχα σχετικά έγγραφα (relevant_docs) στο test_queries:
    β)Καλείται η μέθοδος rank του μοντέλου για να κατατάξει τα έγγραφα για το συγκεκριμένο ερώτημα.
    γ)Δημιουργείται μια λίστα retrieved_docs που περιέχει τα ανακτημένα έγγραφα.

3)Συμπλήρωση Λιστών y_true και y_pred:
Για κάθε έγγραφο (doc_id) στα σχετικά έγγραφα (relevant_docs):

    Προστίθεται 1 στη λίστα y_true αν το έγγραφο είναι σχετικό, αλλιώς 0.
    Προστίθεται 1 στη λίστα y_pred αν το έγγραφο ανακτήθηκε, αλλιώς 0.

4)Υπολογισμός Μετρικών:

    α)Υπολογίζεται η ακρίβεια (precision) χρησιμοποιώντας τη συνάρτηση precision_score.
    β)Υπολογίζεται η ανάκληση (recall) χρησιμοποιώντας τη συνάρτηση recall_score.
    γ)Υπολογίζεται το F1 score χρησιμοποιώντας τη συνάρτηση f1_score.


In [2]:
import requests
import nltk
from bs4 import BeautifulSoup as bs
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re
from collections import defaultdict
import math
from sklearn.metrics import precision_score, recall_score, f1_score
from collections import defaultdict
import math
from sklearn.metrics import precision_score, recall_score, f1_score




class QueryProcessor:
    def __init__(self, data):
        self.data = data

    def tokenize(self, query):
        return re.findall(r'\w+', query.lower())

    def search(self, query):
        tokens = self.tokenize(query)
        results = set(self.data)
        
        if 'and' in tokens:
            and_index = tokens.index('and')
            left_tokens = tokens[:and_index]
            right_tokens = tokens[and_index+1:]
            results = self._and_search(left_tokens, right_tokens)
        elif 'or' in tokens:
            or_index = tokens.index('or')
            left_tokens = tokens[:or_index]
            right_tokens = tokens[or_index+1:]
            results = self._or_search(left_tokens, right_tokens)
        elif 'not' in tokens:
            not_index = tokens.index('not')
            left_tokens = tokens[:not_index]
            right_tokens = tokens[not_index+1:]
            results = self._not_search(left_tokens, right_tokens)
        else:
            results = self._simple_search(tokens)
        
        return results

    def _simple_search(self, tokens):
        return {item for item in self.data if all(token in item.lower() for token in tokens)}

    def _and_search(self, left_tokens, right_tokens):
        left_results = self._simple_search(left_tokens)
        right_results = self._simple_search(right_tokens)
        return left_results & right_results

    def _or_search(self, left_tokens, right_tokens):
        left_results = self._simple_search(left_tokens)
        right_results = self._simple_search(right_tokens)
        return left_results | right_results

    def _not_search(self, left_tokens, right_tokens):
        left_results = self._simple_search(left_tokens)
        right_results = self._simple_search(right_tokens)
        return left_results - right_results

# Create a TF-IDF model
class TFIDF:
    def __init__(self, documents):
        self.documents = documents
        self.idf = self.compute_idf()

    def compute_tf(self, document):
        tf = defaultdict(float)
        for term in document:
            tf[term] += 1
        for term in tf:
            tf[term] /= len(document)
        return tf

    def compute_idf(self):
        idf = defaultdict(float)
        total_documents = len(self.documents)
        for document in self.documents.values():
            for term in set(document):
                idf[term] += 1
        for term in idf:
            idf[term] = math.log(total_documents / (1 + idf[term]))
        return idf

    def compute_tfidf(self, document):
        tf = self.compute_tf(document)
        tfidf = defaultdict(float)
        for term in tf:
            tfidf[term] = tf[term] * self.idf[term]
        return tfidf

    def rank(self, query):
        query_tokens = word_tokenize(query.lower())
        query_tokens = [re.sub(r'\W', '', token) for token in query_tokens if re.sub(r'\W', '', token)]
        query_tokens = [token for token in query_tokens if token not in stop_words]
        query_tokens = [lemmatizer.lemmatize(token) for token in query_tokens]

        query_tfidf = self.compute_tfidf(query_tokens)
        scores = defaultdict(float)

        for doc_id, document in self.documents.items():
            doc_tfidf = self.compute_tfidf(document)
            for term in query_tfidf:
                if term in doc_tfidf:
                    scores[doc_id] += query_tfidf[term] * doc_tfidf[term]

        ranked_docs = sorted(scores.items(), key=lambda item: item[1], reverse=True)
        return ranked_docs
class VectorSpaceModel:
    def __init__(self, documents):
        self.documents = documents
        self.tfidf = TFIDF(documents)

    def compute_cosine_similarity(self, vec1, vec2):
        intersection = set(vec1.keys()) & set(vec2.keys())
        numerator = sum([vec1[x] * vec2[x] for x in intersection])

        sum1 = sum([vec1[x] ** 2 for x in vec1.keys()])
        sum2 = sum([vec2[x] ** 2 for x in vec2.keys()])
        denominator = math.sqrt(sum1) * math.sqrt(sum2)

        if not denominator:
            return 0.0
        else:
            return float(numerator) / denominator

    def rank(self, query):
        query_tokens = word_tokenize(query.lower())
        query_tokens = [re.sub(r'\W', '', token) for token in query_tokens if re.sub(r'\W', '', token)]
        query_tokens = [token for token in query_tokens if token not in stop_words]
        query_tokens = [lemmatizer.lemmatize(token) for token in query_tokens]

        query_tfidf = self.tfidf.compute_tfidf(query_tokens)
        scores = defaultdict(float)

        for doc_id, document in self.documents.items():
            doc_tfidf = self.tfidf.compute_tfidf(document)
            scores[doc_id] = self.compute_cosine_similarity(query_tfidf, doc_tfidf)

        ranked_docs = sorted(scores.items(), key=lambda item: item[1], reverse=True)
        return ranked_docs
class OkapiBM25:
    def __init__(self, documents, k1=1.5, b=0.75):
        self.documents = documents
        self.k1 = k1
        self.b = b
        self.avg_doc_len = self.compute_avg_doc_len()
        self.idf = self.compute_idf()

    def compute_avg_doc_len(self):
        total_length = sum(len(doc) for doc in self.documents.values())
        return total_length / len(self.documents)

    def compute_idf(self):
        idf = defaultdict(float)
        total_documents = len(self.documents)
        for document in self.documents.values():
            for term in set(document):
                idf[term] += 1
        for term in idf:
            idf[term] = math.log((total_documents - idf[term] + 0.5) / (idf[term] + 0.5) + 1)
        return idf

    def compute_bm25(self, document, query_tokens):
        doc_len = len(document)
        score = 0.0
        for term in query_tokens:
            if term in document:
                term_freq = document.count(term)
                numerator = self.idf[term] * term_freq * (self.k1 + 1)
                denominator = term_freq + self.k1 * (1 - self.b + self.b * (doc_len / self.avg_doc_len))
                score += numerator / denominator
        return score

    def rank(self, query):
        query_tokens = word_tokenize(query.lower())
        query_tokens = [re.sub(r'\W', '', token) for token in query_tokens if re.sub(r'\W', '', token)]
        query_tokens = [token for token in query_tokens if token not in stop_words]
        query_tokens = [lemmatizer.lemmatize(token) for token in query_tokens]


        scores = defaultdict(float)
        for doc_id, document in self.documents.items():
            scores[doc_id] = self.compute_bm25(document, query_tokens)
            ranked_docs = sorted(scores.items(), key=lambda item: item[1], reverse=True)
        return ranked_docs

def evaluate_search_engine(model, test_queries):
    y_true = []
    y_pred = []

    for query, relevant_docs in test_queries.items():
        ranked_docs = model.rank(query)
        retrieved_docs = [doc_id for doc_id, _ in ranked_docs]

        for doc_id in relevant_docs:
            y_true.append(1 if doc_id in relevant_docs else 0)
            y_pred.append(1 if doc_id in retrieved_docs else 0)

    precision = precision_score(y_true, y_pred, average='binary')
    recall = recall_score(y_true, y_pred, average='binary')
    f1 = f1_score(y_true, y_pred, average='binary')

    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1}")

#Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

url = "https://en.wikipedia.org/wiki/Enron"
page = requests.get(url)
soup = bs(page.text, 'html.parser')

#Extract text from the main content of the webpage
content = soup.find(id="bodyContent")
text = content.get_text()

#Tokenization
tokens = word_tokenize(text)

#Remove special characters and convert to lowercase
tokens = [re.sub(r'\W', '', token).lower() for token in tokens if re.sub(r'\W', '', token)]

#Remove stop words
stop_words = set(stopwords.words('english'))
tokens = [token for token in tokens if token not in stop_words]

#Stemming
stemmer = PorterStemmer()
stemmed_tokens = [stemmer.stem(token) for token in tokens]

#Lemmatization
lemmatizer = WordNetLemmatizer()
lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

#print("Lemmatized Tokens:", lemmatized_tokens[:100])
#print("Stemmed Tokens:", stemmed_tokens[:100])

# Create inverted index
inverted_index = {}
for index, token in enumerate(lemmatized_tokens):
    if token not in inverted_index:
        inverted_index[token] = []
    inverted_index[token].append(index)
#print("Inverted Index:", {k: v[:5] for k, v in inverted_index.items()})  # Print first 5 entries for brevity
# Save the inverted index to a file
qp =QueryProcessor(inverted_index)
documents = {
    0: inverted_index,
}
tfidf = TFIDF(documents)
vsm = VectorSpaceModel(documents)

bm25 = OkapiBM25(inverted_index)

test_queries = {
    "energy market": [0],
    "financial scandal": [0],
    "email communication": [0],
}

# Evaluate the performance of each model
print("Evaluating TF-IDF Model")
evaluate_search_engine(tfidf, test_queries)

print("Evaluating Vector Space Model")
evaluate_search_engine(vsm, test_queries)

print("Evaluating Okapi BM25 Model")
evaluate_search_engine(bm25, test_queries)



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Costa.000\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Costa.000\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Costa.000\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Evaluating TF-IDF Model
Precision: 1.0
Recall: 1.0
F1-Score: 1.0
Evaluating Vector Space Model
Precision: 1.0
Recall: 1.0
F1-Score: 1.0
Evaluating Okapi BM25 Model
Precision: 0.0
Recall: 0.0
F1-Score: 0.0


c:\Users\Costa.000\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Παραδείγματα Εκτέλεσης:

Τα παρακάτω παραδείγματα αφορούν την επιλογή μεθόδου ανάκτησης αποτελεσμάτων  με βάση τις εισαγωγές ερωτημάτων από τον χρήστη.

Εκτελώντας τον κώδικα εμφανίζεται στην οθόνη του χρήστη το παρακάτω μενού επιλογών:

Choose an option:
1. TF-IDF Ranking
2. Boolean Retrieval
3. Vector Space Model
4. Okapi BM25 Ranking
5. Exit


Enter your choice (1-5): 1                                                                                      
Enter your query for TF-IDF Ranking: enron                                                                      
Ranked documents for 'enron': [(0, 0.00016353063782103517)]

Enter your choice (1-5): 2                                                                                      
Enter your query for Boolean Retrieval: enron                                                                   
Search results for 'enron': {'titleenron', 'enroncom', 'enrononline', 'enron', 'exenron', 'divisionsenron', 'enronmailcom', 'baxterenron', 'vteenron', 'enronthemed', 'enroned', 'enrondefunct', 'xceleratorwebsiteenroncom', 'xcomenronstatus1874921084698640534', 'enroncredit'}

Enter your choice (1-5): 3                                                                                      
Enter your query for Vector Space Model: email                                                                  
Ranked documents for 'email': [(0, 0.018449054095122736)]

Enter your choice (1-5): 4                                                                                      
Enter your query for Okapi BM25 Ranking: scam                                                                   
Ranked documents for 'scam': [('wikipedia', 0.0)

Enter your choice (1-5): 7                                                                                      
Invalid choice. Please try again.

Enter your choice (1-5): 5                                                                                      
Exiting...